In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
import numpy as np

In [2]:
data = pd.read_csv('non_nan.csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [3]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)
    
    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [4]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [5]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [6]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

In [7]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

In [8]:
def collect_prediction(X_train,y_train,X_test,y_test,estimator,alpha,model_name):
  estimator.fit(X_train,y_train)
  y_pred = estimator.predict(X_test)
  #print( "{model_name} alpha = {alpha:.2f},score = {score:.1f}".format(model_name=model_name, alpha=alpha , score= XGBQuantile.quantile_score(y_test, y_pred, alpha)) )

  return y_pred

In [9]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']
dates12m = ['2022-05-01']

In [10]:
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

to_exclude3

['Date',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG']

In [39]:
def rolling_forecast_QR(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []
    all_returns = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude,axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[0:horizon]
        y_test = test_df[ticker][0:horizon]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)
        all_var_forecast.append(VaR_forecast)

        r = y_test
        all_returns.append(r)

        loss = tick_loss(alpha, r, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)
    all_returns = np.concatenate(all_returns)

    return t_loss, all_var_forecast, all_returns

In [12]:
from scipy.stats import norm
from arch import arch_model
import numpy as np

def rolling_forecast_GARCH(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude, axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[horizon-1]
        y_test = test_df[ticker][horizon-1]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=horizon)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)
        all_var_forecast.append(VaR_forecast)

        r = y_test
        loss = tick_loss(alpha, r, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)

    return t_loss, all_var_forecast

In [17]:
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

def rolling_forecast_XGB(data, ticker, alpha, exclude, dates, horizon): #,params, features=False, gcv=False)

    all_var_forecast = []

    t_loss = 0
    dta = data.copy()

    for d in dates:
      split_date = d
      train_df = dta[dta.Date <= split_date]
      train_df = train_df.dropna()
      test_df = dta[dta.Date > split_date]
      test_df = test_df.reset_index(drop=True)

      X_train = train_df.drop(columns=exclude, axis=1)
      X_test = test_df.drop(columns=exclude, axis=1)
      X_test = X_test.iloc[0:horizon]
      y_train = train_df[ticker]
      y_test = test_df[ticker][0:horizon]

      regressor = GradientBoostingRegressor()

      #y_pred = regressor.fit(X_train, y_train).predict(X_test)
      regressor.set_params(loss='quantile', alpha=0.05)
      y_lower = collect_prediction(X_train, y_train, X_test, y_test, estimator=regressor, alpha=0.05, model_name="Gradient Boosting")
      regressor.set_params(loss='quantile', alpha=0.95)
      #y_upper = collect_prediction(X_train, y_train, X_test, y_test, estimator=regressor, alpha=0.95, model_name="Gradient Boosting")
      tickloss = tick_loss(alpha, y_test, y_lower)
      #print(tickloss)
      t_loss += tickloss

      all_var_forecast.append(y_lower)

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)
    return t_loss, all_var_forecast

# 1 month

In [159]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all = []
var_forecast_all =[]
returns_all_1m = []

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=1)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=1)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates1m, horizon=1)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all.append(var_forecast)

    # Store returns for each ticker
    returns_all_1m.append(all_returns)

tick_losses_all = np.array(tick_losses_all)
var_forecast_all = np.array(var_forecast_all)

print("Tick Losses Matrix:")
print(tick_losses_all)
print("VaR Forecasts All:")
print(var_forecast_all)
print("All Returns:")
print(returns_all_1m)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1178.7314788173323
Iteration:      2,   Func. Count:     14,   Neg. LLF: 893.9662722206108
Iteration:      3,   Func. Count:     21,   Neg. LLF: 947.2792115942004
Iteration:      4,   Func. Count:     28,   Neg. LLF: 889.7749000463948
Iteration:      5,   Func. Count:     33,   Neg. LLF: 889.7675789803586
Iteration:      6,   Func. Count:     38,   Neg. LLF: 889.7537238994715
Iteration:      7,   Func. Count:     43,   Neg. LLF: 889.7361309667915
Iteration:      8,   Func. Count:     48,   Neg. LLF: 889.7274260931832
Iteration:      9,   Func. Count:     53,   Neg. LLF: 889.725416391847
Iteration:     10,   Func. Count:     58,   Neg. LLF: 889.7253145862583
Iteration:     11,   Func. Count:     62,   Neg. LLF: 889.7253145862426
Optimization terminated successfully    (Exit mode 0)
            Current function value: 889.7253145862583
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

...

Iteration:      9,   Func. Count:     50,   Neg. LLF: 1026.5286922601586
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterations: 12
            Function evaluations: 65
            Gradient evaluations: 12
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1156.7999856927881
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1075.3769103078384
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1041.1938827489212
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1050.0008570604436
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1048.2288453655756
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1043.837904983452
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1042.2065

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1174.1994644695042
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1135.118426737945
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1056.4270033692883
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1056.305188552166
Iteration:      5,   Func. Count:     30,   Neg. LLF: 1056.322908214031
Iteration:      6,   Func. Count:     36,   Neg. LLF: 1056.4279057832605
Iteration:      7,   Func. Count:     42,   Neg. LLF: 1055.967431020837
Iteration:      8,   Func. Count:     47,   Neg. LLF: 1055.951717995651
Iteration:      9,   Func. Count:     52,   Neg. LLF: 1055.9489931761782
Iteration:     10,   Func. Count:     57,   Neg. LLF: 1055.948763654921
Iteration:     11,   Func. Count:     62,   Neg. LLF: 15680.335044552896
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1055.9487633497965
            Iterations: 12
            Function evaluations: 68
            Gradient evaluatio

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Tick Losses Matrix:
[[[0.89672138 0.75102131 0.76209529]]

 [[0.71467371 0.48696721 0.49613247]]

 [[2.31440119 0.46835748 0.60501453]]

 [[1.36682505 0.69379176 0.67465423]]

 [[0.65496638 0.58647176 0.62188408]]

 [[0.7442595  0.66693109 0.6294229 ]]

 [[1.49964732 0.45892824 0.51904428]]

 [[2.31069231 0.77356387 0.86559629]]

 [[4.4995135  3.98913715 4.08063691]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.06133914  -8.89037173  -7.37917121]
  [ -3.47685705  -8.66274921  -7.99221562]
  [ -9.30813624  -9.12655521  -7.98912243]
  [-12.5588184  -10.64292001 -10.44553367]
  [ -7.23167789 -10.35228813  -8.39829201]
  [ -7.97638183 -12.05033945  -7.65436944]
  [-15.34285013 -10.84558587  -8.23273473]
  [-14.46254905 -10.29198289  -8.01119484]
  [ -1.77012818  -9.69677902  -8.03051215]
  [ -7.88142787  -9.81479006  -7.91620285]
  [-19.04914778  -9.55508869  -8.42191174]]

 [[ -9.43470816  -9.73720522 -10.38596559]
  [-10.86444788  -9.61872766  -9.98262098]
  [ -8

In [19]:
tick_losses_all = np.array(tick_losses_all)
tick_losses_all_matrix = np.reshape(tick_losses_all, (9, 3))
tick_losses_all_matrix

array([[0.89672138, 0.75102131, 0.76263606],
       [0.71467371, 0.48696721, 0.48461207],
       [2.31440119, 0.46835748, 0.63329696],
       [1.36682505, 0.69379176, 0.67465467],
       [0.65496638, 0.58647176, 0.58185675],
       [0.7442595 , 0.66693109, 0.6294229 ],
       [1.49964732, 0.45892824, 0.51814113],
       [2.31069231, 0.77356387, 0.86391333],
       [4.4995135 , 3.98913715, 4.03904542]])

In [20]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker = []
for tick_losses in tick_losses_all_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker.append(weights)

# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.29675168 0.35432228 0.34892604]
Weights for Ticker 2: [0.25365678 0.37226703 0.37407619]
Weights for Ticker 3: [0.10420949 0.51495404 0.38083646]
Weights for Ticker 4: [0.20015844 0.39432807 0.40551349]
Weights for Ticker 5: [0.30840973 0.34442921 0.34716106]
Weights for Ticker 6: [0.30317784 0.33833029 0.35849187]
Weights for Ticker 7: [0.13962581 0.45625753 0.40411666]
Weights for Ticker 8: [0.15011061 0.44839145 0.40149794]
Weights for Ticker 9: [0.30845674 0.34792117 0.3436221 ]


In [160]:
weights_array = np.array(weights_per_ticker)
weights_array

array([[0.29675168, 0.35432228, 0.34892604],
       [0.25365678, 0.37226703, 0.37407619],
       [0.10420949, 0.51495404, 0.38083646],
       [0.20015844, 0.39432807, 0.40551349],
       [0.30840973, 0.34442921, 0.34716106],
       [0.30317784, 0.33833029, 0.35849187],
       [0.13962581, 0.45625753, 0.40411666],
       [0.15011061, 0.44839145, 0.40149794],
       [0.30845674, 0.34792117, 0.3436221 ]])

In [161]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts = []
row_sums_per_ticker = []
for ticker in range(len(var_forecast_all)):
    weights = weights_array[ticker]
    var_matrix = var_forecast_all[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts.append(weighted_var_matrix)
    row_sums = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker.append(row_sums)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums) in enumerate(zip(weighted_var_forecasts, row_sums_per_ticker)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.91922973 -3.24273986 -2.97258491]
 [-0.61170586 -3.15005676 -2.57478498]
 [-1.03176318 -3.06940503 -2.78869214]
 [-2.7622051  -3.23374183 -2.78761284]
 [-3.72685051 -3.77102366 -3.64471868]
 [-2.14601259 -3.66804631 -2.93038276]
 [-2.36700474 -4.26970372 -2.67080881]
 [-4.55301661 -3.84283269 -2.87261551]
 [-4.29178579 -3.64667882 -2.79531448]
 [-0.52528852 -3.43578483 -2.80205479]
 [-2.33882699 -3.47759877 -2.7621693 ]
 [-5.65286668 -3.38558079 -2.9386243 ]]
Row sums for Ticker 1:
[ -9.13455449  -6.3365476   -6.88986035  -8.78355977 -11.14259285
  -8.74444166  -9.30751727 -11.26846482 -10.73377908  -6.76312814
  -8.57859506 -11.97707177]
Weighted VaR Forecast Matrix for Ticker 2:
[[-2.39317773 -3.62484044 -3.88514243]
 [-2.7558409  -3.58073515 -3.73426082]
 [-2.16405042 -3.68456448 -4.11335267]
 [ 0.38017873 -4.16479915 -4.10811584]
 [-3.84197067 -3.89777599 -4.10542376]
 [-4.11361396 -4.06736948 -4.10321522]
 [-4.75361911 -3.80229192 -4

In [162]:
## Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for t, (weighted_var_matrix, row_sums) in enumerate(zip(weighted_var_forecasts, row_sums_per_ticker)):
    print(f"Ensemble forecasts for Ticker {t+1}:")
    print(row_sums)

Ensemble forecasts for Ticker 1:
[ -9.13455449  -6.3365476   -6.88986035  -8.78355977 -11.14259285
  -8.74444166  -9.30751727 -11.26846482 -10.73377908  -6.76312814
  -8.57859506 -11.97707177]
Ensemble forecasts for Ticker 2:
[ -9.9031606  -10.07083687  -9.96196758  -7.89273626 -11.84517042
 -12.28419866 -12.65682748 -11.77057622  -8.50343279 -11.39667567
 -12.73631161  -9.75957337]
Ensemble forecasts for Ticker 3:
[-5.42171857 -5.33265531 -4.29595761 -6.42723011 -6.41126559 -7.62116756
 -5.04375961 -6.82387728 -6.92142956 -7.43899991 -7.25317603 -7.24821616]
Ensemble forecasts for Ticker 4:
[ -9.2280587   -9.70864855  -7.35239847  -8.14599633  -8.71886426
  -9.34981005  -8.54384007  -8.45688228  -9.10979516  -7.84370591
  -9.39630411 -10.35956231]
Ensemble forecasts for Ticker 5:
[ -8.30733295 -13.6675989  -13.50888185  -6.99745221  -9.04968526
 -11.20492653 -13.70777609 -12.16352356 -10.5676892   -8.44875851
  -7.71891852  -9.69977276]
Ensemble forecasts for Ticker 6:
[ -8.67238643 -

In [163]:
for t in range(len(row_sums_per_ticker)):
    return_ticker = row_sums_per_ticker[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker)

Ensemble forecasts for Ticker 1: [ -9.13455449  -6.3365476   -6.88986035  -8.78355977 -11.14259285
  -8.74444166  -9.30751727 -11.26846482 -10.73377908  -6.76312814
  -8.57859506 -11.97707177]
Ensemble forecasts for Ticker 2: [ -9.9031606  -10.07083687  -9.96196758  -7.89273626 -11.84517042
 -12.28419866 -12.65682748 -11.77057622  -8.50343279 -11.39667567
 -12.73631161  -9.75957337]
Ensemble forecasts for Ticker 3: [-5.42171857 -5.33265531 -4.29595761 -6.42723011 -6.41126559 -7.62116756
 -5.04375961 -6.82387728 -6.92142956 -7.43899991 -7.25317603 -7.24821616]
Ensemble forecasts for Ticker 4: [ -9.2280587   -9.70864855  -7.35239847  -8.14599633  -8.71886426
  -9.34981005  -8.54384007  -8.45688228  -9.10979516  -7.84370591
  -9.39630411 -10.35956231]
Ensemble forecasts for Ticker 5: [ -8.30733295 -13.6675989  -13.50888185  -6.99745221  -9.04968526
 -11.20492653 -13.70777609 -12.16352356 -10.5676892   -8.44875851
  -7.71891852  -9.69977276]
Ensemble forecasts for Ticker 6: [ -8.67238643 -

In [164]:
return_ticker.flatten()

array([-10.57169003, -25.5084965 , -24.68919821, -16.74438534,
       -17.66845939, -16.48671977, -29.38844209, -10.81336612,
       -21.06053861, -23.54722187, -18.08736394, -13.6838466 ])

In [165]:
Var_list = []
for t in range(len(row_sums_per_ticker)):
    return_ticker = row_sums_per_ticker[t]
    Var_list.append(return_ticker)

In [168]:
ret_list = returns_all_1m

In [169]:
tick_loss_list=[]
for i in range(len(Var_list)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list[i], var=Var_list[i])
    tick_loss_list.append(tick_loss_ensemble)

In [170]:
tick_loss_list_1m=[]
for i in range(len(tick_loss_list)):
    tick_loss_list_1m.append(tick_loss_list[i]/12)

tick_loss_list_1m

[0.7403488954799328,
 0.49146655988056814,
 0.6002671320730073,
 0.7408904109340266,
 0.6198899424490281,
 0.6583464792358977,
 0.5603004712171626,
 0.9707344013152409,
 3.940595266104072]

# 3 months

In [91]:
returns_all_3m = []
tick_losses_all_3m = []
var_forecast_all_3m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates3m, horizon=3)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_3m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_3m.append(var_forecast)

    # Store returns for each ticker
    returns_all_3m.append(all_returns)

tick_losses_all_3m = np.array(tick_losses_all_3m)
var_forecast_all_3m = np.array(var_forecast_all_3m)

print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)
print("All Returns:")
print(returns_all_3m)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      3,   Func. Count:     20,   Neg. LLF: 878.1339497577962
Iteration:      4,   Func. Count:     27,   Neg. LLF: 876.6737053132299
Iteration:      5,   Func. Count:     32,   Neg. LLF: 876.6672585066991
Iteration:      6,   Func. Count:     37,   Neg. LLF: 876.6384410362984
Iteration:      7,   Func. Count:     42,   Neg. LLF: 876.589180523503
Iteration:      8,   Func. Count:     47,   Neg. LLF: 876.587517374675
Iteration:      9,   Func. Count:     52,   Neg. LLF: 876.5873820200592
Iteration:     10,   Func. Count:     57,   Neg. LLF: 876.5873810789516
Optimization terminated successfully    (Exit mode 0)
            Current function value: 876.5873810789516
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1222.9902668269392
Iteration:      2,   Func. Count:     13,   Neg. LLF: 886.8361595084345
Iteration:      3,   Func. Count:     19,   Neg. LLF: 884.5076379612233


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1233.4593893444899
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1028.629179221663
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1026.5330869611244
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1026.5334575056986
Iteration:      5,   Func. Count:     30,   Neg. LLF: 1026.5321650488156
Iteration:      6,   Func. Count:     35,   Neg. LLF: 1026.5321236507464
Iteration:      7,   Func. Count:     40,   Neg. LLF: 1026.531915967955
Iteration:      8,   Func. Count:     45,   Neg. LLF: 1026.5310091930328
Iteration:      9,   Func. Count:     50,   Neg. LLF: 1026.5286922601586
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterati

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Tick Losses Matrix:
[[[ 2.95079751  2.42349257  2.5143962 ]]

 [[ 2.28026507  1.45078368  1.48440117]]

 [[ 6.78132128  1.38240144  1.58913646]]

 [[ 4.46836782  2.05583184  1.94185975]]

 [[ 2.11000197  1.85296406  1.87442112]]

 [[ 2.24409617  1.95704452  1.98626   ]]

 [[ 4.74213431  1.44374109  1.55548345]]

 [[ 8.07806017  2.25544276  2.98830157]]

 [[14.08557797 13.54669229 13.92983042]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.75467099]
  [ -2.06133914  -8.89037173  -7.25280725]
  [ -3.47686179  -9.12229136  -6.14293289]
  [-10.29957562  -9.29385125  -7.25280725]
  [ -3.47685705  -8.66274921  -8.0186908 ]
  [-10.29963214  -8.94963189  -8.11201201]
  [-11.69836103  -9.16075273  -7.77434563]
  [ -9.30813624  -9.12655521  -7.98915856]
  [-12.65689204  -9.30123411  -7.75428972]
  [ -7.54087514  -9.43113514  -7.84249292]
  [-12.5588184  -10.64292001 -10.22654282]
  [ -7.44888531 -10.4701

In [83]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all_3m = []
var_forecast_all_3m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates3m, horizon=3)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_3m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_3m.append(var_forecast)

    #returns of the tickers
    all_returns_3m = all_returns

    print(f'{t}-3m_return: {all_returns_3m} \n')

tick_losses_all_3m = np.array(tick_losses_all_3m)
var_forecast_all_3m = np.array(var_forecast_all_3m)


print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      3,   Func. Count:     19,   Neg. LLF: 888.0315247874007
Iteration:      4,   Func. Count:     26,   Neg. LLF: 884.6907414212512
Iteration:      5,   Func. Count:     31,   Neg. LLF: 884.6848259256694
Iteration:      6,   Func. Count:     36,   Neg. LLF: 884.6574834856478
Iteration:      7,   Func. Count:     41,   Neg. LLF: 884.6397555584947
Iteration:      8,   Func. Count:     46,   Neg. LLF: 884.6286696410642
Iteration:      9,   Func. Count:     51,   Neg. LLF: 884.6273420337402
Iteration:     10,   Func. Count:     56,   Neg. LLF: 884.6273335106139
Iteration:     11,   Func. Count:     60,   Neg. LLF: 884.6273335108109
Optimization terminated successfully    (Exit mode 0)
            Current function value: 884.6273335106139
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1178.7314788173323
Iteration:      2,   Func. Count:     14,   Neg. LLF: 893.966272220610

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1233.4593893444899
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1028.629179221663
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1026.5330869611244
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1026.5334575056986
Iteration:      5,   Func. Count:     30,   Neg. LLF: 1026.5321650488156
Iteration:      6,   Func. Count:     35,   Neg. LLF: 1026.5321236507464
Iteration:      7,   Func. Count:     40,   Neg. LLF: 1026.531915967955
Iteration:      8,   Func. Count:     45,   Neg. LLF: 1026.5310091930328
Iteration:      9,   Func. Count:     50,   Neg. LLF: 1026.5286922601586
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterati

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

ROG-3m_return: [ -1.2397349    2.73188743  -6.95635867   2.73188743  -6.95635867
  -3.44882614  -6.95635867  -3.44882614  -2.70795556  -3.44882614
  -2.70795556 -53.66506605  -2.70795556 -53.66506605   9.44607056
 -53.66506605   9.44607056  16.96832631   9.44607056  16.96832631
   5.4516805   16.96832631   5.4516805   11.02581254   5.4516805
  11.02581254  -1.51746671  11.02581254  -1.51746671  -2.16215955] 

Tick Losses Matrix:
[[[ 2.95079751  2.42349257  2.5074638 ]]

 [[ 2.28026507  1.45078368  1.49889415]]

 [[ 6.78132128  1.38240144  1.56464648]]

 [[ 4.46836782  2.05583184  1.94186423]]

 [[ 2.11000197  1.85296406  1.87442109]]

 [[ 2.24409617  1.95704452  1.98626   ]]

 [[ 4.74213431  1.44374109  1.5544907 ]]

 [[ 8.07806017  2.25544276  3.02881592]]

 [[14.08557797 13.54669229 13.90074562]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.51924071]
  [ -2.06133914  -8.89037173  -7.37917609

In [93]:
tick_losses_all_3m = np.array(tick_losses_all_3m)
tick_losses_all_3m_matrix = np.reshape(tick_losses_all_3m, (9, 3))
tick_losses_all_3m_matrix

array([[ 2.95079751,  2.42349257,  2.5143962 ],
       [ 2.28026507,  1.45078368,  1.48440117],
       [ 6.78132128,  1.38240144,  1.58913646],
       [ 4.46836782,  2.05583184,  1.94185975],
       [ 2.11000197,  1.85296406,  1.87442112],
       [ 2.24409617,  1.95704452,  1.98626   ],
       [ 4.74213431,  1.44374109,  1.55548345],
       [ 8.07806017,  2.25544276,  2.98830157],
       [14.08557797, 13.54669229, 13.92983042]])

In [94]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_3m = []
for tick_losses in tick_losses_all_3m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_3m.append(weights)

In [95]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_3m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.29488594 0.3590474  0.34606666]
Weights for Ticker 2: [0.24343347 0.38261585 0.37395068]
Weights for Ticker 3: [0.0983017 0.4822155 0.4194828]
Weights for Ticker 4: [0.18266225 0.39701794 0.42031981]
Weights for Ticker 5: [0.30633509 0.34882903 0.34483588]
Weights for Ticker 6: [0.30520503 0.34997131 0.34482366]
Weights for Ticker 7: [0.13636478 0.44790587 0.41572934]
Weights for Ticker 8: [0.13727188 0.49165093 0.37107718]
Weights for Ticker 9: [0.32776561 0.34080408 0.33143031]


In [96]:
weights_array_3m = np.array(weights_per_ticker_3m)

In [97]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_3m = []
row_sums_per_ticker_3m = []
for ticker in range(len(var_forecast_all_3m)):
    weights = weights_array_3m[ticker]
    var_matrix = var_forecast_all_3m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_3m.append(weighted_var_matrix)
    row_sums_3m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_3m.append(row_sums_3m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_3m) in enumerate(zip(weighted_var_forecasts_3m, row_sums_per_ticker_3m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_3m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.9008759  -3.28598391 -2.94822519]
 [-0.77797036 -3.34636191 -2.80906515]
 [-0.21443844 -3.39130411 -3.02969976]
 [-0.60785993 -3.19206483 -2.50995479]
 [-1.02527766 -3.27533497 -2.12586428]
 [-3.03720005 -3.3369331  -2.50995479]
 [-1.02527626 -3.11033756 -2.77500155]
 [-3.03721672 -3.21334204 -2.80729692]
 [-3.4496822  -3.28914442 -2.69044184]
 [-2.74483852 -3.27686589 -2.76478143]
 [-3.73233953 -3.3395839  -2.68350116]
 [-2.22369806 -3.38622452 -2.71402534]
 [-3.70341899 -3.82131273 -3.53906553]
 [-2.19657156 -3.7592751  -3.38734336]
 [-1.7539033  -3.71222604 -3.38734336]
 [-2.13252014 -3.71696211 -2.8123571 ]
 [-1.74234397 -3.67815159 -2.8123571 ]
 [-4.67818709 -3.64884116 -2.99130431]
 [-2.35212287 -4.32664301 -2.73691412]
 [-5.32550844 -4.14753474 -2.92574244]
 [-4.34876052 -4.00961823 -2.73691412]
 [-4.5243908  -3.89407938 -2.85675084]
 [-4.65862574 -3.81362562 -2.66906185]
 [-0.63511496 -3.75262373 -2.66906185]
 [-4.26480239 -3.6953

In [50]:
for t in range(len(row_sums_per_ticker_3m)):
    return_ticker_3m = row_sums_per_ticker_3m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_3m)

Ensemble forecasts for Ticker 1: [ -9.12876352  -6.94554805  -6.57414076  -6.39273068  -7.40879144
  -9.12417187  -7.0027645   -9.15679768  -9.48541328  -8.82079929
  -9.79835165  -8.35086269 -11.24815101  -9.66913568  -9.07018871
  -8.74364353  -8.31906027 -11.39062556  -9.39475404 -12.40117324
 -11.05712612 -11.32182374 -11.17540523  -7.13281903 -10.65708873
  -6.68572653  -8.78259587  -6.75109138  -8.57161083 -11.96091436]
Ensemble forecasts for Ticker 2: [ -9.87632805 -10.45251938  -9.95784277 -10.43423715 -10.11041008
  -7.70769028  -9.97071704  -7.6466145  -11.85663271  -8.00772311
 -11.98676467 -11.56435143 -11.54943235 -11.3474976  -12.55946598
 -12.23637841 -13.03117339 -11.61849733 -12.58442922 -11.27668533
  -8.62010995 -11.60947479  -8.72298449 -11.05266743  -8.43131489
 -10.92237799 -12.62157179 -11.36570283 -12.98032971 -10.14061562]
Ensemble forecasts for Ticker 3: [-5.61176736 -5.70679013 -5.10931723 -5.332926   -4.63716516 -5.90403222
 -4.35438622 -6.17921965 -6.030325

In [98]:
return_ticker_3m.flatten()

array([-10.07960619, -23.76763118, -26.84322784, -26.09546835,
       -26.06652241, -17.75772599, -25.2618579 , -17.85457442,
       -16.69837633, -16.83779393, -16.84244307, -16.87246605,
       -17.76192024, -17.79503199, -11.62949607, -16.62286195,
       -12.81664294, -11.92174797, -28.82848918, -23.01251225,
       -33.16285632, -10.31654572, -20.86378053, -22.21531568,
       -21.28467117, -22.47176354, -18.54740563, -23.79467365,
       -18.61753185, -12.71070211])

In [102]:
Var_list_3m = []
for t in range(len(row_sums_per_ticker_3m)):
    return_ticker_3m = row_sums_per_ticker_3m[t]
    Var_list_3m.append(return_ticker_3m)

ret_list_3m = all_returns_3m

tick_loss_list_3=[]
for i in range(len(Var_list_3m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_3m[i], var=Var_list_3m[i])
    tick_loss_list_3.append(tick_loss_ensemble)

tick_loss_list_3m=[]
for i in range(len(tick_loss_list_3)):
    tick_loss_list_3m.append(tick_loss_list_3[i]/10)

tick_loss_list_3m

[1.1630379971728158,
 2.0165897968760893,
 2.162281347299481,
 1.7234712167169028,
 0.5201842483481148,
 0.8908778458989233,
 0.5754657784271782,
 0.9827902935411746,
 2.518058355535223]

# 6 months

In [104]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
returns_all_6m = []
tick_losses_all_6m = []
var_forecast_all_6m =[]


for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=6)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=6)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates6m, horizon=6)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_6m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_6m.append(var_forecast)

    # Store returns for each ticker
    returns_all_6m.append(all_returns)

tick_losses_all_6m = np.array(tick_losses_all_6m)
var_forecast_all_6m = np.array(var_forecast_all_6m)


print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)
print("All Returns:")
print(returns_all_3m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      3,   Func. Count:     19,   Neg. LLF: 884.5076379612233
Iteration:      4,   Func. Count:     26,   Neg. LLF: 881.4156102171411
Iteration:      5,   Func. Count:     31,   Neg. LLF: 881.4116765176805
Iteration:      6,   Func. Count:     36,   Neg. LLF: 881.3924911968852
Iteration:      7,   Func. Count:     41,   Neg. LLF: 881.3541270434575
Iteration:      8,   Func. Count:     46,   Neg. LLF: 881.353888375146
Iteration:      9,   Func. Count:     51,   Neg. LLF: 881.3538747215401
Iteration:     10,   Func. Count:     55,   Neg. LLF: 881.3538747213574
Optimization terminated successfully    (Exit mode 0)
            Current function value: 881.3538747215401
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1227.934884202285
Iteration:      2,   Func. Count:     13,   Neg. LLF: 889.2892608331276
Iteration:      3,   Func. Count:     19,   Neg. LLF: 888.0315247874007


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      4,   Func. Count:     28,   Neg. LLF: 889.7749000463948
Iteration:      5,   Func. Count:     33,   Neg. LLF: 889.7675789803586
Iteration:      6,   Func. Count:     38,   Neg. LLF: 889.7537238994715
Iteration:      7,   Func. Count:     43,   Neg. LLF: 889.7361309667915
Iteration:      8,   Func. Count:     48,   Neg. LLF: 889.7274260931832
Iteration:      9,   Func. Count:     53,   Neg. LLF: 889.725416391847
Iteration:     10,   Func. Count:     58,   Neg. LLF: 889.7253145862583
Iteration:     11,   Func. Count:     62,   Neg. LLF: 889.7253145862426
Optimization terminated successfully    (Exit mode 0)
            Current function value: 889.7253145862583
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11


...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      7,   Func. Count:     40,   Neg. LLF: 1026.531915967955
Iteration:      8,   Func. Count:     45,   Neg. LLF: 1026.5310091930328
Iteration:      9,   Func. Count:     50,   Neg. LLF: 1026.5286922601586
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterations: 12
            Function evaluations: 65
            Gradient evaluations: 12
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1156.7999856927881
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1075.3769103078384
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1041.1938827489212
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1050.0008570604436
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1048.2288

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Tick Losses Matrix:
[[[ 2.95079751  2.42349257  2.5143962 ]]

 [[ 2.28026507  1.45078368  1.48440117]]

 [[ 6.78132128  1.38240144  1.58913646]]

 [[ 4.46836782  2.05583184  1.94185975]]

 [[ 2.11000197  1.85296406  1.87442112]]

 [[ 2.24409617  1.95704452  1.98626   ]]

 [[ 4.74213431  1.44374109  1.55548345]]

 [[ 8.07806017  2.25544276  2.98830157]]

 [[14.08557797 13.54669229 13.92983042]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.75467099]
  [ -2.06133914  -8.89037173  -7.25280725]
  [ -3.47686179  -9.12229136  -6.14293289]
  [-10.29957562  -9.29385125  -7.25280725]
  [ -3.47685705  -8.66274921  -8.0186908 ]
  [-10.29963214  -8.94963189  -8.11201201]
  [-11.69836103  -9.16075273  -7.77434563]
  [ -9.30813624  -9.12655521  -7.98915856]
  [-12.65689204  -9.30123411  -7.75428972]
  [ -7.54087514  -9.43113514  -7.84249292]
  [-12.5588184  -10.64292001 -10.22654282]
  [ -7.44888531 -10.4701

In [105]:
tick_losses_all_6m = np.array(tick_losses_all_6m)
tick_losses_all_6m_matrix = np.reshape(tick_losses_all_6m, (9, 3))
tick_losses_all_6m_matrix

array([[ 5.43825096,  4.01877127,  5.20775413],
       [ 4.45719481,  2.66627322,  3.15969878],
       [10.36730275,  2.47600197,  2.8602457 ],
       [ 7.09199951,  4.06037535,  3.76148828],
       [ 4.54496316,  3.36714969,  3.98642658],
       [ 4.72937822,  4.58264163,  4.1661157 ],
       [ 8.68946218,  3.3159731 ,  3.14232854],
       [13.82873898,  4.89922753,  6.53214035],
       [35.38864232, 35.95314858, 37.40602061]])

In [106]:

# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_6m = []
for tick_losses in tick_losses_all_6m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_6m.append(weights)

In [107]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_6m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.29433644 0.3982997  0.30736386]
Weights for Ticker 2: [0.24495792 0.40949485 0.34554723]
Weights for Ticker 3: [0.11348505 0.47517483 0.41134012]
Weights for Ticker 4: [0.21588666 0.3770755  0.40703784]
Weights for Ticker 5: [0.28654075 0.38677139 0.32668786]
Weights for Ticker 6: [0.31573421 0.32584404 0.35842175]
Weights for Ticker 7: [0.15659793 0.41036274 0.43303932]
Weights for Ticker 8: [0.16835961 0.475218   0.35642239]
Weights for Ticker 9: [0.34125419 0.33589611 0.3228497 ]


In [108]:
weights_array_6m = np.array(weights_per_ticker_6m)

In [109]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_6m = []
row_sums_per_ticker_6m = []
for ticker in range(len(var_forecast_all_6m)):
    weights = weights_array_6m[ticker]
    var_matrix = var_forecast_all_6m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_6m.append(weighted_var_matrix)
    row_sums_6m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_6m.append(row_sums_6m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_6m) in enumerate(zip(weighted_var_forecasts_6m, row_sums_per_ticker_6m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_6m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.89547026 -3.64521908 -2.6185067 ]
 [-0.77652065 -3.71219781 -2.4949098 ]
 [-0.21403885 -3.76205324 -2.69086946]
 [-3.63234332 -3.79930267 -2.4949098 ]
 [-3.57205057 -3.82720922 -2.69086946]
 [-1.84329397 -3.84815782 -2.6185067 ]
 [-0.60672722 -3.54103243 -2.27728128]
 [-1.02336711 -3.63340595 -2.45963846]
 [-3.03154038 -3.70173821 -2.27728128]
 [-3.44326449 -3.75256124 -2.3980769 ]
 [-2.10632707 -3.79050751 -2.60121917]
 [-1.44675483 -3.81891847 -2.45963846]
 [-1.02336571 -3.45037045 -2.57817384]
 [-3.03155702 -3.56463574 -2.27203014]
 [-3.4432539  -3.64872511 -2.24001115]
 [-2.10632097 -3.71104108 -2.42134468]
 [-1.44676264 -3.75744755 -2.42134468]
 [-4.99077952 -3.79212699 -2.57050695]
 [-2.73972365 -3.63510424 -2.45563616]
 [-3.7253845  -3.7046788  -2.4612102 ]
 [-2.21955431 -3.75641834 -2.53132945]
 [-2.16612682 -3.79504656 -2.38974874]
 [-4.33821509 -3.82396801 -2.60816141]
 [-4.36824801 -3.84566683 -2.38974874]
 [-3.69651785 -4.2390

In [124]:
return_ticker_6m
for t in range(len(row_sums_per_ticker_6m)):
    return_ticker_6m = row_sums_per_ticker_6m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_6m)

Ensemble forecasts for Ticker 1: [ -9.15919604  -6.98362826  -6.66696155  -9.9265558  -10.09012926
  -8.30995849  -6.42504093  -7.11641152  -9.01055987  -9.59390263
  -8.49805375  -7.72531175  -7.05191001  -8.8682229   -9.33199015
  -8.23870673  -7.62555486 -11.35341346  -8.83046405  -9.89127349
  -8.50730211  -8.35092212 -10.77034451 -10.60366358 -11.04442754
  -9.42843548  -8.93439948 -11.62622299 -11.8564196   -7.6462488
  -8.65126816  -8.21876563 -11.2413114  -11.13866514  -7.3130042
  -9.01036768  -9.59808703 -12.48218003 -11.23932102  -7.69806709
  -9.45251514 -13.26230366]
Ensemble forecasts for Ticker 2: [-10.11174004 -10.70309844 -10.21409925  -7.90984723 -11.91645518
 -11.5856099  -10.39979954 -10.09128212  -7.69233615 -11.87625185
 -11.60299818 -12.8118246   -9.59246154  -7.27723887 -11.49787625
 -11.24119385 -12.45758789 -11.12093277  -8.01162335 -11.99100353
 -11.56590791 -12.83834472 -11.4702406   -8.72809668 -11.79010327
 -11.59236007 -12.80788518 -11.47421776  -8.733525

In [111]:
return_ticker_6m.flatten()

array([ -9.90669856, -23.47262528, -27.23506707, -16.66464589,
       -17.06638533, -17.51179922, -26.16518885, -26.48793325,
       -17.7767471 , -17.18967048, -17.26534434, -17.25990793,
       -25.69818239, -17.84838422, -16.67900337, -16.87095929,
       -12.68753022, -11.90351738, -16.88589955, -16.85064688,
       -16.86883305, -12.57661006, -11.79614291, -18.77271845,
       -17.73066962, -17.82695113, -11.02600645, -11.59673309,
       -19.03333351, -22.00109078, -16.61979366, -12.64401432,
       -11.72151277, -18.8469952 , -21.84325045, -15.37933619,
       -28.41153161, -22.35255073, -32.92629682, -34.47550503,
       -30.71779319, -23.48091141])

In [129]:
Var_list_6m = []
for t in range(len(row_sums_per_ticker_6m)):
    return_ticker_6m = row_sums_per_ticker_6m[t]
    Var_list_6m.append(return_ticker_6m)

ret_list_6m = returns_all_6m

tick_loss_list_6=[]
for i in range(len(Var_list_6m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_6m[i], var=Var_list_6m[i])
    tick_loss_list_6.append(tick_loss_ensemble)

tick_loss_list_6m=[]
for i in range(len(tick_loss_list_6)):
    tick_loss_list_6m.append(tick_loss_list_6[i]/7)

tick_loss_list_6m

[4.670570101968272,
 3.1223017770396098,
 3.0903848572303887,
 3.7987517626606446,
 4.0953600554667196,
 4.314262356334645,
 3.519577006577877,
 6.491129675802621,
 35.74174025674068]

# 9 months

In [134]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
returns_all_9m = []
tick_losses_all_9m = []
var_forecast_all_9m =[]


for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=9)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=9)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates9m, horizon=9)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_9m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_9m.append(var_forecast)

    # Store returns for each ticker
    returns_all_9m.append(all_returns)

tick_losses_all_9m = np.array(tick_losses_all_9m)
var_forecast_all_9m = np.array(var_forecast_all_9m)


print("Tick Losses Matrix:")
print(tick_losses_all_9m)
print("VaR Forecasts All:")
print(var_forecast_all_9m)
print("All Returns:")
print(returns_all_9m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      3,   Func. Count:     19,   Neg. LLF: 904.0742102389393
Iteration:      4,   Func. Count:     26,   Neg. LLF: 888.0176227402903
Iteration:      5,   Func. Count:     31,   Neg. LLF: 888.0170831290263
Iteration:      6,   Func. Count:     36,   Neg. LLF: 888.0170406502615
Iteration:      7,   Func. Count:     41,   Neg. LLF: 888.0167817047684
Iteration:      8,   Func. Count:     46,   Neg. LLF: 888.0154865930697
Iteration:      9,   Func. Count:     51,   Neg. LLF: 888.0109869234884
Iteration:     10,   Func. Count:     56,   Neg. LLF: 888.009594587629
Iteration:     11,   Func. Count:     61,   Neg. LLF: 888.0095017604818
Iteration:     12,   Func. Count:     66,   Neg. LLF: 888.0094967912892
Iteration:     13,   Func. Count:     70,   Neg. LLF: 888.0094967911114
Optimization terminated successfully    (Exit mode 0)
            Current function value: 888.0094967912892
            Iterations: 13
            Function evaluations: 70
            Gradient evaluations: 13


...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 867.9900312422881
Iteration:      2,   Func. Count:     13,   Neg. LLF: 859.2616055493777
Iteration:      3,   Func. Count:     20,   Neg. LLF: 843.7600474146132
Iteration:      4,   Func. Count:     27,   Neg. LLF: 833.3095140110556
Iteration:      5,   Func. Count:     32,   Neg. LLF: 833.230974403957
Iteration:      6,   Func. Count:     37,   Neg. LLF: 834.5977092957178
Iteration:      7,   Func. Count:     43,   Neg. LLF: 833.2088013618945
Iteration:      8,   Func. Count:     49,   Neg. LLF: 833.1985189269656
Iteration:      9,   Func. Count:     54,   Neg. LLF: 833.1985119125833
Iteration:     10,   Func. Count:     58,   Neg. LLF: 833.1985119122887
Optimization terminated successfully    (Exit mode 0)
            Current function value: 833.1985119125833
            Iterations: 10
            Function evaluations: 58
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 922.0800563778539


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Tick Losses Matrix:
[[[ 8.18461796  5.72395866  7.72745367]]

 [[ 6.93617375  3.64972198  4.61993721]]

 [[20.60742827  2.50038446  5.3916736 ]]

 [[13.67942271  7.56808716  5.8064445 ]]

 [[ 6.42922845  4.68162065  5.72570982]]

 [[ 6.89263536  5.91161015  5.94513772]]

 [[14.61166592  4.7635035   4.61255105]]

 [[16.81566142  3.80591614  9.01868665]]

 [[45.10503871  8.94014037 43.49466677]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.51924071]
  [-12.34078718  -9.53880365  -8.11712154]
  [-12.13594422  -9.60886785  -8.51924071]
  [ -6.26254089  -9.6614629   -8.51924071]
  [ -2.7415266   -9.70100222  -8.51924071]
  [-20.55341384  -9.73075894  -8.85001669]
  [-14.06886188  -9.75317152  -8.51924071]
  [ -2.06133914  -8.89037173  -7.48202166]
  [ -3.47686179  -9.12229136  -8.30599373]
  [-10.29957562  -9.29385125  -7.48202166]
  [-11.69839703  -9.42145122  -8.11957112]
  [ -7.15618868  -9.5167

In [135]:
tick_losses_all_9m = np.array(tick_losses_all_9m)
tick_losses_all_9m_matrix = np.reshape(tick_losses_all_9m, (9, 3))
tick_losses_all_9m_matrix

array([[ 8.18461796,  5.72395866,  7.72745367],
       [ 6.93617375,  3.64972198,  4.61993721],
       [20.60742827,  2.50038446,  5.3916736 ],
       [13.67942271,  7.56808716,  5.8064445 ],
       [ 6.42922845,  4.68162065,  5.72570982],
       [ 6.89263536,  5.91161015,  5.94513772],
       [14.61166592,  4.7635035 ,  4.61255105],
       [16.81566142,  3.80591614,  9.01868665],
       [45.10503871,  8.94014037, 43.49466677]])

In [136]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_9m = []
for tick_losses in tick_losses_all_9m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_9m.append(weights)

In [137]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_9m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.28661107 0.40982164 0.30356729]
Weights for Ticker 2: [0.2271786  0.43174528 0.34107612]
Weights for Ticker 3: [0.07654748 0.63088165 0.29257088]
Weights for Ticker 4: [0.19367019 0.35006155 0.45626826]
Weights for Ticker 5: [0.28602799 0.39279972 0.32117228]
Weights for Ticker 6: [0.30072272 0.35062732 0.34864996]
Weights for Ticker 7: [0.13821269 0.42395636 0.43783095]
Weights for Ticker 8: [0.13730925 0.60667279 0.25601796]
Weights for Ticker 9: [0.14119803 0.71237614 0.14642583]


In [138]:
weights_array_9m = np.array(weights_per_ticker_9m)

In [139]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_9m = []
row_sums_per_ticker_9m = []
for ticker in range(len(var_forecast_all_9m)):
    weights = weights_array_9m[ticker]
    var_matrix = var_forecast_all_9m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_9m.append(weighted_var_matrix)
    row_sums_9m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_9m.append(row_sums_9m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_9m) in enumerate(zip(weighted_var_forecasts_9m, row_sums_per_ticker_9m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_9m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.81947369 -3.75066727 -2.58616278]
 [-0.75613954 -3.81958355 -2.46409255]
 [-0.20842103 -3.87088119 -2.58616278]
 [-3.53700625 -3.90920817 -2.46409255]
 [-3.47829599 -3.937922   -2.58616278]
 [-1.79491356 -3.95947659 -2.58616278]
 [-0.78575188 -3.97568066 -2.58616278]
 [-5.89083599 -3.9878756  -2.68657555]
 [-4.0322916  -3.99706076 -2.58616278]
 [-0.59080262 -3.64346674 -2.27129701]
 [-0.99650709 -3.73851242 -2.52142797]
 [-2.95197242 -3.80882138 -2.27129701]
 [-3.35289012 -3.8611146  -2.46483616]
 [-2.05104291 -3.90015858 -2.55982914]
 [-1.40878227 -3.9293914  -2.44719236]
 [-4.85979543 -3.95132298 -2.60529432]
 [-3.94893043 -3.96780152 -2.52142797]
 [-0.38289286 -3.9801966  -2.32788882]
 [-0.99650573 -3.5501821  -2.54045245]
 [-2.95198862 -3.66775283 -2.42523626]
 [-3.3528798  -3.75427472 -2.36046837]
 [-2.05103697 -3.81839336 -2.38625718]
 [-1.40878987 -3.86614227 -2.38625718]
 [-4.85978797 -3.90182491 -2.50560443]
 [-3.94893754 -3.9285

In [140]:
for t in range(len(row_sums_per_ticker_9m)):
    return_ticker_9m = row_sums_per_ticker_9m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_9m)

Ensemble forecasts for Ticker 1: [ -9.15630374  -7.03981564  -6.665465    -9.91030698 -10.00238077
  -8.34055293  -7.34759531 -12.56528714 -10.61551513  -6.50556637
  -7.25644748  -9.0320908   -9.67884089  -8.51103064  -7.78536603
 -11.41641273 -10.43815993  -6.69097827  -7.08714028  -9.04497771
  -9.4676229   -8.25568752  -7.66118932 -11.26721731 -10.41794793
  -6.71776488  -8.51750698  -8.93055512  -9.84324571  -8.38718226
  -8.37490271 -10.70786756 -10.57131006  -6.61798852  -8.54653114
 -11.75186722]
Ensemble forecasts for Ticker 2: [ -9.87727122 -10.45634312 -10.01622853  -7.88520558 -11.60370762
 -11.29809692 -12.52430605 -11.31197116  -8.62053928 -10.37149341
 -10.11874518  -7.90843971 -11.79515444 -11.54464716 -12.66705267
 -11.42478936  -8.86988149 -10.95547279  -9.63782275  -7.51578984
 -11.44122178 -11.20818614 -12.33857041 -11.09996861  -8.54674688
 -10.63311402 -12.31264149  -8.23543662 -11.91592466 -11.51713358
 -12.69517233 -11.42545184  -8.88192665 -11.023471   -12.6571

In [141]:
return_ticker_9m.flatten()

array([-14.01313465, -19.34421294, -21.27497166, -16.89562428,
       -17.00480947, -17.33741372, -15.71918808, -14.36280538,
       -18.16045827, -19.50258636, -20.60444257, -17.12714838,
       -16.94821997, -16.95850178, -15.97271776, -15.07599291,
       -17.93215705, -19.19532556, -20.03025393, -16.92370637,
       -16.47172042, -16.55645343, -14.90232389, -14.62373751,
       -17.63680718, -18.81680808, -16.20545523, -16.38233753,
       -16.46309257, -16.53982386, -14.81237418, -14.54278514,
       -17.45752024, -18.6691209 , -16.16708155, -13.77742707])

In [148]:
Var_list_9m = []
for t in range(len(row_sums_per_ticker_9m)):
    return_ticker_9m = row_sums_per_ticker_9m[t]
    Var_list_9m.append(return_ticker_9m)

ret_list_9m = returns_all_9m

tick_loss_list_9=[]
for i in range(len(Var_list_9m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_9m[i], var=Var_list_9m[i])
    tick_loss_list_9.append(tick_loss_ensemble)

tick_loss_list_9m=[]
for i in range(len(tick_loss_list_9)):
    tick_loss_list_9m.append(tick_loss_list_9[i]/4)

tick_loss_list_9m

[6.828578313581044,
 4.540983768368021,
 4.865333251018035,
 6.333552827486317,
 5.828059331576308,
 6.128528547790986,
 5.219576489182321,
 7.43755117853509,
 43.18977743793615]

# 12 months

In [149]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
returns_all_12m = []
tick_losses_all_12m = []
var_forecast_all_12m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=12)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=12)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates12m, horizon=12)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_12m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_12m.append(var_forecast)

    # Store returns for each ticker
    returns_all_12m.append(all_returns)

tick_losses_all_12m = np.array(tick_losses_all_12m)
var_forecast_all_12m = np.array(var_forecast_all_12m)


print("Tick Losses Matrix:")
print(tick_losses_all_12m)
print("VaR Forecasts All:")
print(var_forecast_all_12m)
print("All Returns:")
print(returns_all_12m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 3737.673074947868
Iteration:      2,   Func. Count:     14,   Neg. LLF: 1059.3458407887865
Iteration:      3,   Func. Count:     20,   Neg. LLF: 990.3442313922687
Iteration:      4,   Func. Count:     26,   Neg. LLF: 774.7400448569347
Iteration:      5,   Func. Count:     32,   Neg. LLF: 779.060699049972
Iteration:      6,   Func. Count:     38,   Neg. LLF: 771.3832501664328
Iteration:      7,   Func. Count:     44,   Neg. LLF: 769.7415752103974
Iteration:      8,   Func. Count:     50,   Neg. LLF: 765.6335026311029
Iteration:      9,   Func. Count:     55,   Neg. LLF: 765.6297294381544
Iteration:     10,   Func. Count:     60,   Neg. LLF: 765.6289482309151
Iteration:     11,   Func. Count:     65,   Neg. LLF: 765.628625130922
Iteration:     12,   Func. Count:     70,   Neg. LLF: 765.6283320636917
Iteration:     13,   Func. Count:     75,   Neg. LLF: 765.6283000831261
Iteration:     14,   Func. Count:     80,   Neg. LLF: 765.6282984

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 867.9900312422881
Iteration:      2,   Func. Count:     13,   Neg. LLF: 859.2616055493777
Iteration:      3,   Func. Count:     20,   Neg. LLF: 843.7600474146132
Iteration:      4,   Func. Count:     27,   Neg. LLF: 833.3095140110556
Iteration:      5,   Func. Count:     32,   Neg. LLF: 833.230974403957
Iteration:      6,   Func. Count:     37,   Neg. LLF: 834.5977092957178
Iteration:      7,   Func. Count:     43,   Neg. LLF: 833.2088013618945
Iteration:      8,   Func. Count:     49,   Neg. LLF: 833.1985189269656
Iteration:      9,   Func. Count:     54,   Neg. LLF: 833.1985119125833
Iteration:     10,   Func. Count:     58,   Neg. LLF: 833.1985119122887
Optimization terminated successfully    (Exit mode 0)
            Current function value: 833.1985119125833
            Iterations: 10
            Function evaluations: 58
            Gradient evaluations: 10


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Tick Losses Matrix:
[[[11.20032919  5.64811163  9.13538468]]

 [[ 8.44300384  4.76410303  5.69613706]]

 [[27.74149871  0.78862259  5.72429897]]

 [[19.24636627 11.73234121  8.00652069]]

 [[ 7.84057241  3.18186983  7.65407857]]

 [[ 9.00189077  3.12873027  7.42138368]]

 [[21.74261694  0.61556187  6.13896092]]

 [[18.61012257  6.18731604 10.42144804]]

 [[54.87740592  8.93592537 47.07984382]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.51924071]
  [-12.34078718  -9.53880365  -8.11712154]
  [-12.13594422  -9.60886785  -8.51924071]
  [ -6.26254089  -9.6614629   -8.51924071]
  [ -2.7415266   -9.70100222  -8.51924071]
  [-20.55341384  -9.73075894  -8.85001669]
  [-14.06886188  -9.75317152  -8.51924071]
  [  1.35556108  -9.77006275  -8.11712154]
  [ -7.29180996  -9.78279858  -8.51924071]
  [-20.67749581  -9.79240455  -8.51924071]]

 [[ -9.43470816  -9.73720522 -10.34642368]
  [-10.8634487  -10.32

In [150]:
tick_losses_all_12m = np.array(tick_losses_all_12m)
tick_losses_all_12m_matrix = np.reshape(tick_losses_all_12m, (9, 3))
tick_losses_all_12m_matrix

array([[11.20032919,  5.64811163,  9.13538468],
       [ 8.44300384,  4.76410303,  5.69613706],
       [27.74149871,  0.78862259,  5.72429897],
       [19.24636627, 11.73234121,  8.00652069],
       [ 7.84057241,  3.18186983,  7.65407857],
       [ 9.00189077,  3.12873027,  7.42138368],
       [21.74261694,  0.61556187,  6.13896092],
       [18.61012257,  6.18731604, 10.42144804],
       [54.87740592,  8.93592537, 47.07984382]])

In [151]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_12m = []
for tick_losses in tick_losses_all_12m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_12m.append(weights)

In [152]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_12m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.23758274 0.47113178 0.29128548]
Weights for Ticker 2: [0.23504823 0.41655546 0.34839631]
Weights for Ticker 3: [0.02437632 0.85748947 0.11813421]
Weights for Ticker 4: [0.19824394 0.32521006 0.476546  ]
Weights for Ticker 5: [0.22279132 0.54898898 0.2282197 ]
Weights for Ticker 6: [0.19645836 0.56524421 0.23829743]
Weights for Ticker 7: [0.02508572 0.88606714 0.08884714]
Weights for Ticker 8: [0.1726061 0.5191622 0.3082317]
Weights for Ticker 9: [0.12038279 0.73929612 0.14032109]


In [153]:
weights_array_12m = np.array(weights_per_ticker_12m)

In [154]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_12m = []
row_sums_per_ticker_12m = []
for ticker in range(len(var_forecast_all_12m)):
    weights = weights_array_12m[ticker]
    var_matrix = var_forecast_all_12m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_12m.append(weighted_var_matrix)
    row_sums_12m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_12m.append(row_sums_12m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_12m) in enumerate(zip(weighted_var_forecasts_12m, row_sums_per_ticker_12m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_12m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.3371682  -4.31177462 -2.4815311 ]
 [-0.62679261 -4.39100092 -2.36439963]
 [-0.17276806 -4.44997279 -2.4815311 ]
 [-2.93195803 -4.49403356 -2.36439963]
 [-2.88329088 -4.52704303 -2.4815311 ]
 [-1.48787162 -4.55182223 -2.4815311 ]
 [-0.6513394  -4.57045046 -2.4815311 ]
 [-4.88313637 -4.5844698  -2.57788135]
 [-3.34251875 -4.59502908 -2.4815311 ]
 [ 0.32205792 -4.60298707 -2.36439963]
 [-1.73240819 -4.60898733 -2.4815311 ]
 [-4.91261611 -4.613513   -2.4815311 ]]
Row sums for Ticker 1:
[ -9.13047392  -7.38219316  -7.10427196  -9.79039122  -9.89186501
  -8.52122496  -7.70332097 -12.04548752 -10.41907893  -6.64532879
  -8.82292662 -12.00766022]
Weighted VaR Forecast Matrix for Ticker 2:
[[-2.21761147 -4.05608599 -3.60465582]
 [-2.55343441 -4.30162497 -3.60465582]
 [-1.98296503 -4.4089657  -3.60465582]
 [ 0.27330998 -4.45692668 -3.60465582]
 [-3.55081304 -4.47855228 -3.60465582]
 [-3.22411733 -4.48834222 -3.60465582]
 [-4.48804209 -4.49278206 -3

In [155]:
for t in range(len(row_sums_per_ticker_12m)):
    return_ticker_12m = row_sums_per_ticker_12m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_12m)

Ensemble forecasts for Ticker 1: [ -9.13047392  -7.38219316  -7.10427196  -9.79039122  -9.89186501
  -8.52122496  -7.70332097 -12.04548752 -10.41907893  -6.64532879
  -8.82292662 -12.00766022]
Ensemble forecasts for Ticker 2: [ -9.87835328 -10.4597152   -9.99658655  -7.78827252 -11.63402114
 -11.31711537 -12.58547997 -11.33100308  -8.54627184 -10.86187867
 -12.4717635   -9.42731243]
Ensemble forecasts for Ticker 3: [-6.41407366 -6.42292059 -6.25982158 -6.58844356 -6.56667537 -6.6207931
 -6.51858672 -6.78456713 -6.44613162 -6.59317342 -6.68451277 -6.74665237]
Ensemble forecasts for Ticker 4: [ -9.14116465 -10.02406075  -7.64725521  -9.08630413  -8.25792014
  -9.88937391  -8.77645091  -9.78542946  -8.87086699  -7.53034554
  -8.98880492 -11.02545316]
Ensemble forecasts for Ticker 5: [ -8.68565173 -12.59518007 -12.58606166  -7.93930142  -9.31426348
 -10.89458443 -12.95060903 -10.56301744 -10.28683297  -8.91538019
  -8.34174956  -9.90161205]
Ensemble forecasts for Ticker 6: [ -8.76326009 -1

In [156]:
return_ticker_12m.flatten()

array([-14.40575623, -20.15862901, -20.62960635, -17.00099877,
       -16.9674235 , -17.24030411, -17.0220872 , -14.73706382,
       -17.98388793, -19.01859478, -16.67991172, -14.69660344])

In [158]:
Var_list_12m = []
for t in range(len(row_sums_per_ticker_12m)):
    return_ticker_12m = row_sums_per_ticker_12m[t]
    Var_list_12m.append(return_ticker_12m)

ret_list_12m = returns_all_12m

tick_loss_list_12=[]
for i in range(len(Var_list_12m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_12m[i], var=Var_list_12m[i])
    tick_loss_list_12.append(tick_loss_ensemble)

tick_loss_list_12m=[]
for i in range(len(tick_loss_list_12)):
    tick_loss_list_12m.append(tick_loss_list_12[i]/1)

tick_loss_list_12m

[7.867560817002641,
 5.773414020268544,
 5.398996691793685,
 8.975234543457645,
 7.33527569372992,
 7.743395546962186,
 5.2469287407436935,
 8.697901192668931,
 45.44811576992854]